In [1]:
# Globals to override
PROJCET_ID = ""
LOCATION_ID = "us-central1"
AGENT_ID = ""
LANGUAGE_CODE = 'en' 
#-us'    

## Functions Code

In [2]:
from google.cloud import dialogflowcx_v3beta1 as dialogflow #cx
import uuid
from google.cloud.dialogflowcx_v3beta1.services.agents import AgentsClient
from google.cloud.dialogflowcx_v3beta1.services.sessions import SessionsClient
from google.cloud.dialogflowcx_v3beta1.types import session


In [3]:
def say(texts = ["Balance"],session_id = uuid.uuid4(),project_id=PROJCET_ID,location_id=LOCATION_ID,agent_id=AGENT_ID):
    agent = f"projects/{project_id}/locations/{location_id}/agents/{agent_id}"
    language_code = LANGUAGE_CODE

    responses = detect_intent_texts(agent, session_id, texts, language_code)
    for response in responses:
        print(f"Query text: {response.query_result.text}")
        response_messages = [
            " ".join(msg.text.text) for msg in response.query_result.response_messages
        ]
        print(f"Response text: {response_messages}\n")
    return session_id

In [4]:
def detect_intent_texts(agent, session_id, texts, language_code):
    """Returns the result of detect intent with texts as inputs.

    Using the same `session_id` between requests allows continuation
    of the conversation."""
    responses = []
    session_path = f"{agent}/sessions/{session_id}"
    #print(f"Session path: {session_path}\n")
    client_options = None
    agent_components = AgentsClient.parse_agent_path(agent)
    location_id = agent_components["location"]
    if location_id != "global":
        api_endpoint = f"{location_id}-dialogflow.googleapis.com:443"
        print(f"API Endpoint: {api_endpoint}\n")
        client_options = {"api_endpoint": api_endpoint}
    session_client = SessionsClient(client_options=client_options)

    texts = [texts] if isinstance(texts,str) else texts
    for text in texts:
        text_input = session.TextInput(text=text)
        query_input = session.QueryInput(text=text_input, language_code=language_code)
        request = session.DetectIntentRequest(
            session=session_path, query_input=query_input
        )
        response = session_client.detect_intent(request=request)
        responses.append(response)
    return responses



## Embeded Bot

In [ ]:
html = f'''<script src="https://www.gstatic.com/dialogflow-console/fast/messenger-cx/bootstrap.js?v=1"></script>
<df-messenger
  df-cx="true"
  chat-title="Test Bot"
  agent-id="{AGENT_ID}"
  language-code="{LANGUAGE_CODE}"
></df-messenger>'''
from IPython.core.display import display, HTML
display(HTML(html))

## Dialog

In [5]:
session_id = say("Credit card balance amount")

Query text: Credit card balance amount
Response text: ["I'd be happy to help with your account balance.\n", "\nWhat are the last four digits of the card that you're calling about?", '']



In [6]:
session_id

UUID('f7929111-2dcd-46ed-8bb7-5cae13ff6e5e')

In [7]:
texts = ["again, I'm asking for a balance check" \
         ,"can I speak with your manager" \
         ,"Can you help with payments?"]
say(texts,session_id)

ResourceExhausted: 429 received initial metadata size exceeds limit